In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize 

In [28]:
df = pd.read_csv("/Users/Administrator/PycharmProjects/pythonProject2/PytonProject/netflix_titles.csv")
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [29]:
#create a new feature that we will use to calculate the similarity between movies, it's a combination of some other features
df["overall_infos"]= df["type"] + " "+df["title"]+" "+df["description"]

In [30]:
#adding new column because there are mistakes in show_id column
df.insert(1, "id", list(range(1, len(df)+1)), True) 

In [31]:
df.head()

,show_id,id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,overall_infos
0,s1,1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,TV Show 3% In a future where the elite inhabit...
1,s2,2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Movie 7:19 After a devastating earthquake hits...
2,s3,3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Movie 23:59 When an army recruit is found dead...
3,s4,4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...","Movie 9 In a postapocalyptic world, rag-doll r..."
4,s5,5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,Movie 21 A brilliant group of students become ...


In [32]:
#
df_processed = df[['id','overall_infos']]
df_processed.head(1)

,id,overall_infos
0,1,TV Show 3% In a future where the elite inhabit...


In [33]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [34]:
#processing the overall info column
def text_preprocessing(column):
    #make all words with lower letters
    column = column.str.lower()
    #getting rid of any punctution
    column = column.str.replace('http\S+|www.\S+|@|%|:|,|', '', case=False)
    #spliting each sentence to words to apply previous funtions on them 
    word_tokens = column.str.split()
    keywords = word_tokens.apply(lambda x: [item for item in x if item not in stop])
    #assemble words of each sentence again and assign them in new column
    for i in range(len(keywords)):
        keywords[i] = " ".join(keywords[i])
        column = keywords

    return column

In [35]:
df_processed['processed_infos'] = text_preprocessing(df_processed['overall_infos'])

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
df_processed.head(1)

,id,overall_infos,processed_infos
0,1,TV Show 3% In a future where the elite inhabit...,tv show 3 future elite inhabit island paradise...


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CV = CountVectorizer()
converted_metrix = CV.fit_transform(df_processed['processed_infos'])

In [38]:
cosine_similarity = cosine_similarity(converted_metrix)

In [39]:
cosine_similarity

array([[1.        , 0.        , 0.05892557, ..., 0.        , 0.10660036,
        0.        ],
       [0.        , 1.        , 0.0571662 , ..., 0.17149859, 0.        ,
        0.05057217],
       [0.05892557, 0.0571662 , 1.        , ..., 0.05555556, 0.        ,
        0.04914732],
       ...,
       [0.        , 0.17149859, 0.05555556, ..., 1.        , 0.        ,
        0.09829464],
       [0.10660036, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.05057217, 0.04914732, ..., 0.09829464, 0.        ,
        1.        ]])

In [40]:
#recommendation based on title, type, description
def get_recommendations(title):    
    if title is not None:
        recom = df[df['overall_infos'].str.contains(title)]
    return recom

In [42]:
title = input("Enter title name: ")
get_recommendations(title)

Enter title name: comedy


,show_id,id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,overall_infos
29,s30,30,TV Show,#blackAF,NaN,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,"April 17, 2020",2020,TV-MA,1 Season,TV Comedies,Kenya Barris and his family navigate relations...,TV Show #blackAF Kenya Barris and his family n...
136,s137,137,Movie,7 Khoon Maaf,Vishal Bhardwaj,"Priyanka Chopra, Neil Nitin Mukesh, John Abrah...",India,"August 2, 2018",2011,TV-MA,148 min,"Dramas, International Movies, Thrillers","Spiced liberally with black comedy, this Bolly...",Movie 7 Khoon Maaf Spiced liberally with black...
174,s175,175,Movie,A Futile and Stupid Gesture,David Wain,"Will Forte, Domhnall Gleeson, Martin Mull, Joe...",United States,"January 26, 2018",2018,TV-MA,102 min,Comedies,"In a brief life full of triumph and failure, ""...",Movie A Futile and Stupid Gesture In a brief l...
210,s211,211,Movie,A Night at the Roxbury,John Fortenberry,"Will Ferrell, Chris Kattan, Dan Hedaya, Molly ...",United States,"December 1, 2019",1998,PG-13,82 min,"Comedies, Cult Movies","After a run-in with Richard Grieco, dimwits Do...",Movie A Night at the Roxbury After a run-in wi...
307,s308,308,Movie,ADAM SANDLER 100% FRESH,Steve Brill,Adam Sandler,United States,"October 23, 2018",2018,TV-MA,74 min,Stand-Up Comedy,"From ""Heroes"" to ""Ice Cream Ladies"" – Adam San...","Movie ADAM SANDLER 100% FRESH From ""Heroes"" to..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7271,s7272,7272,TV Show,Turbo FAST,NaN,"Reid Scott, John Eric Bentley, Amir Talai, Phi...",United States,"February 5, 2016",2016,TV-Y7,3 Seasons,Kids' TV,"Join Turbo and his posse, the Fast Action Stun...","TV Show Turbo FAST Join Turbo and his posse, t..."
7450,s7451,7451,TV Show,W/ Bob & David,NaN,"David Cross, Bob Odenkirk, John Ennis, Jay Joh...",United States,"November 13, 2015",2015,TV-MA,1 Season,TV Comedies,After being dishonorably discharged from the N...,TV Show W/ Bob & David After being dishonorabl...
7477,s7478,7478,Movie,War Chhod Na Yaar,Faraz Haider,"Sharman Joshi, Soha Ali Khan, Javed Jaffrey, D...",India,"March 2, 2019",2013,TV-14,109 min,"Comedies, Dramas, International Movies","Set in an India-Pakistan border post, this sat...",Movie War Chhod Na Yaar Set in an India-Pakist...
7565,s7566,7566,Movie,What the Jatt!!,Saket Behl,"Harish Verma, Isha Rikhi, Vipul Roy, Binnu Dhi...",India,"October 1, 2018",2015,TV-PG,140 min,"Comedies, International Movies, Romantic Movies",A peasant from Punjab and his best friend pose...,Movie What the Jatt!! A peasant from Punjab an...


In [43]:
#this how we will get the id of the movie so we can check similarity between it and other movies
title = 'Teenage Mutant Ninja Turtles'

movie_id = df[df['title'] == title]['id'].values[0]

In [44]:
#simil
score = list(enumerate(cosine_similarity[movie_id]))

In [45]:
#now sort the similar movies in descending order
sorted_score = sorted(score, key=lambda x:x[1], reverse= True)
#we will ignore the first score because it will give us a 100% score because it's the same movie 
sorted_score = sorted_score[1:]

In [46]:
#now showing the top 5 movies similar to TMNT accroding to this algorithm
i = 0
for item in sorted_score:
    movie_title = df[df['id'] == item[0]]['title'].values[0]
    print(i+1,movie_title)
    i = i+1
    if i > 4:
        break

1 Teenage Mutant Ninja Turtles II: The Secret of the Ooze
2 Teenage Cocktail
3 Ninja Hattori
4 Power Rangers Ninja Steel
5 Power Rangers Mystic Force
